# 📌 Uygulama – 3  
# 🔄 `cv::Mat → torch::Tensor` Dönüşümü  
(OpenCV → LibTorch Preprocess Adımı)

---

## 🎯 1) Amaç — *Bu projede ne yaptım?*

Bu uygulamada amacım:

> **OpenCV ile okuduğum bir `cv::Mat` görüntüsünü, PyTorch/LibTorch için uygun formata (`torch::Tensor`) dönüştürmek.**

Yani:

- `test.jpg` → `cv::Mat`  
- `cv::Mat` → `float32`, 0–1 aralığı  
- HWC (Height, Width, Channel) → NCHW (N, Channel, Height, Width)  
- Sonuç: **Modele direkt verebileceğim bir input tensörü.**

---

## 🧩 2) Ana Adımlar — *Kodda yaptığım işlemler*

### 1️⃣ Resmi diskte okumak

```cpp
cv::Mat img = cv::imread("test.jpg");
if (img.empty())
{
    std::cerr << "[HATA] test.jpg yuklenemedi..." << std::endl;
    return -1;
}
std::cout << "[OK] Orijinal resim boyutu: "
          << img.cols << "x" << img.rows << " (W x H)" << std::endl;
```

* img artık BGR formatında, uint8 tipinde bir cv::Mat


### 2️⃣ BGR → RGB dönüşümü
```cpp
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
```


OpenCV varsayılan olarak BGR kullanıyor.

* PyTorch modelleri genelde RGB beklediği için kanalları çeviriyorum.

### 3️⃣ uint8 → float32 ve 0–1 normalize etme
```cpp
cv::Mat img_float;
img.convertTo(img_float, CV_32F, 1.0 / 255.0);
```


* Piksel aralığı: 0–255 → 0–1

* Veri tipi: uint8 → float32

* Çoğu derin öğrenme modeli bu formatı bekliyor

### 4️⃣ from_blob ile cv::Mat → torch::Tensor (NHWC)
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,                               // veri pointer'ı
    {1, img_float.rows, img_float.cols, 3},       // [N, H, W, C]
    torch::TensorOptions().dtype(torch::kFloat32) // float32
);
```


Burada tensörün shape’i: [1, H, W, C] yani NHWC.

N=1 → batch size (tek resim)

H, W → img_float’ın boyutları

C=3 → RGB kanalları

#### 💡 from_blob şunu yapıyor:
**“Bana halihazırda var olan bir bellek bloğunu (OpenCV’nin datası) tensör olarak göster.”**

Yani:

* Bu tensör OpenCV belleğine bağlı.

* img_float yok olursa, tensörün işaret ettiği veri de geçersiz olur.

### 5️⃣ NHWC → NCHW dönüşümü (PyTorch standardı)
```cpp
tensor_image = tensor_image
    .permute({0, 3, 1, 2})  // [N, C, H, W]
    .contiguous();
```


PyTorch modelleri genelde NCHW formatında input bekler:

* N → Batch

* C → Kanal sayısı (3 → RGB)

* H, W → Yükseklik, genişlik

permute({0,3,1,2}) ile:

* [N, H, W, C] → [N, C, H, W]

contiguous():

* Permute sonrası bellek düzenini düzeltiyorum,

Tensör bellekte sıralı (contiguous) halde hizalanıyor.

### 6️⃣ Gerekirse clone ile bağımsız kopya alınabilir

Kod içinde not:
```bash
// from_blob, OpenCV'nin bellek alanını kullanıyor.
// img/img_float yaşam döngüsü bitmeden bu tensörü kullanmalıyım.
// Gerekirse .clone() ile kendi kopyasını oluşturabilirim:
//
// tensor_image = tensor_image.clone();
```


Eğer tensörü modelden sonra da saklamak veya OpenCV tarafını serbest bırakmak istiyorsam:

* clone() ile LibTorch kendi kopyasını oluşturuyor.

* Böylece tensör OpenCV’den bağımsız hale geliyor.

---

## 🧠 3) Bu projede öğrendiğim temel kavramlar

📌 cv::Mat → torch::Tensor dönüşümü

📌 BGR → RGB renk uzayı dönüşümü

📌 uint8 → float32 dönüşümü ve 0–1 scale mantığı

📌 torch::from_blob ile dış bellekten tensör oluşturma

📌 NHWC → NCHW dönüşümü için permute kullanımı

📌 contiguous() ve gerektiğinde clone() kullanma

📌 Model input formatını kontrol etme alışkanlığı

Bu proje, LibTorch inference için en kritik preprocess katmanını oturttu.

----

## ⚠️ 4) Dikkat edilmesi gereken noktalar

from_blob kopya oluşturmaz, var olan belleği sarar.

Bu yüzden img_float hala yaşıyor olmalı.

**Eğer OpenCV tarafını serbest bırakmak istiyorsam → clone() kullanmalıyım.**

Tensor shape’i:

* Modeller genelde [N, 3, H, W] bekler.

* Yanlış axis sırası, saçmalayan sonuçlara veya runtime hatalarına yol açar.

* Normalizasyon (0–1) yapmazsam model performansı bozulabilir.

----
## 🚀 5) Bir Sonraki Aşamada Ne Yapabilirim?

* Aynı tensörü gerçekten bir TorchScript modeline input olarak verebilirim.

Resize ekleyip:

* Modelin beklediği sabit input boyutunu (ör. 64×64, 224×224, 640×640) sağlayabilirim.

Bu kodu:

* Webcam frame’leri için kullanabilirim.

* YOLO / CNN modellerine entegre edebilirim.

----

## 🔖 6) Etiketler / Kategori

Preprocess

cv::Mat → Tensor

from_blob

NHWC → NCHW

LibTorch + OpenCV

Model Input Hazırlama

----
## 📝 7) Tek Cümlelik Özet

OpenCV ile okuduğum test.jpg görüntüsünü, BGR→RGB, normalize, NHWC→NCHW adımlarından geçirip torch::Tensor haline getirerek LibTorch modelleri için kullanılabilir bir input formatına dönüştürdüm.